In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
from IPython.display import Audio

import os
import re

In [3]:
path = '/content/drive/MyDrive/self_education/UML/project_music_denoising/data'

In [4]:
path

'/content/drive/MyDrive/self_education/UML/project_music_denoising/data'

Собираем все в один трек без шума

In [ ]:
loading_path = os.path.join(path, 'raw_tracks')
saving_path = os.path.join(path, 'mix_tracks')

функция, которая принимает путь до трека и название трека (название папки, где лежат отдельные дорожки) и УДАЛЯЕТ ненужные файлы (в ридми.тхт только лицензионная информация, треки с '._' пока не понимаю зачем нужны)

In [ ]:
def remove_trash(path, track):
  load_path = os.path.join(path, track)
  print('trash removed from: ' + load_path)
  os.chdir(path)

  # удаляем треки с '._'
  pattern = f".*{re.escape('._')}.*"
  matching_files = [file for file in os.listdir() if re.search(pattern, file)]
  for file in matching_files:
      file_path = os.path.join(load_path, file)
      os.remove(file_path)

  # удаляем readme.txt
  readme_path = os.path.join(load_path, 'Readme.txt')
  if os.path.exists(readme_path):
    os.remove(readme_path)

функция, которая принимает путь до трека и название трека (название папки, где лежат отдельные дорожки) и СОБИРАЕТ ЕГО В ОДНУ ДОРОЖКУ, а затем сохраняет в папке mix_tracks

In [ ]:
def track_mix(path, track, save_path):
  load_path = os.path.join(path, track)
  print('processing track from here: ' + load_path)
  os.chdir(load_path)

  # собираем все в один трек
  file_names = os.listdir()
  # print(file_names)
  for i in range(len(file_names) - 1):
    if i < 1:
      file_path_1 = os.path.join(load_path, file_names[i])
      file_path_2 = os.path.join(load_path, file_names[i+1])
      audio1 = AudioSegment.from_wav(file_path_1)
      audio2 = AudioSegment.from_wav(file_path_2)
      mixed_audio = audio1.overlay(audio2)
    else:
      audio_next = AudioSegment.from_wav(os.path.join(load_path, file_names[i+1]))
      mixed_audio = mixed_audio.overlay(audio_next)

  # сохраняем в папку mix_tracks под названием трека (работает пока странно, через раз, еще не разобралась. треки иногда появляются с задержкой по времени)

  mixed_audio.export(os.path.join(save_path, track), format="wav")
  # return mixed_audio # можно включить, тогда в коллабе появится возможность прослушать собранный трек


собираю функцию, которая соберет все песни, которые есть в директории

In [ ]:
def mix_all_tracks(load_path, save_path):
  os.chdir(load_path)
  contents = os.listdir()
  print(load_path, contents)

  # можно сделать кастомный track_names, перечислив списком названия треков, которые хочешь собрать

  # убираем лишнее
  track_names = os.listdir()
  for track in track_names:
    remove_trash(load_path, track)

  # объединяем треки
  track_names = os.listdir()
  for track in track_names:
    track_mix(load_path, track, save_path)

  # возможно стоит все таки по отдельности пользоваться


In [ ]:
mix_all_tracks(loading_path, saving_path)